In [ ]:
%reset

In [ ]:
class Individual(object):
    
    
    def __init__(self,Chromosome=0,Fitness=0):
        self.Chromosome=Chromosome
        self.Fitness=Fitness
    
    def Create(self,dim,R):   # create chromosome of dim elements in the range of a and b
        import numpy as np
        self.Chromosome=np.empty([1,dim])
        for i in range(0,dim):
            self.Chromosome[0,i]=(R[i,1]-R[i,0])*np.random.rand()+R[i,0]
    
    def Crossover(self,other):
        import numpy as np
        child1=Individual()
        child2=Individual()
        alpha=np.random.rand()
        child1.Chromosome=(alpha)*(self.Chromosome)+(1-alpha)*(other.Chromosome)
        child2.Chromosome=(1-alpha)*(self.Chromosome)+(alpha)*(other.Chromosome)
        return child1, child2
        
    
    def Mutation(self,R,dim):
        import numpy as np
        import random
        
        mutant=self
        d=random.randint(0,dim-1)
        #d=np.random.random_integers(0,dim-1)  #dim_of_mutation=
        mut_range=0.1*(R[d,1]-R[d,0])
        self.Chromosome[0,d]= max(R[d,0]  , min (R[d,1], self.Chromosome[0,d]+mut_range*(np.random.rand()-0.5) )  )
        return self 
    
    def Dissimilarity(self,other):
        import numpy as np
        return np.linalg.norm(self.Chromosome-other.Chromosome)
        
        
    def contribution_diversity(self,index_cmin,Population,PopSize,case):
        import numpy as np
        
        if case=="Regualr":
            cd=np.empty(shape=[1, PopSize]) 
            for i in range(0,PopSize):#1
                #print(i)
                cd[0,i]=self.Dissimilarity(Population[i])
                
                #cd[0,index_of_self]=np.inf
            #cd[0,PopSize]=np.inf
            
        if case=="cmin-removed": 
            cd=np.empty(shape=[1, PopSize-1]) 
            pop_cmin_removed=Population[:]  #deep copy
            #print(int(index_cmin))
            del pop_cmin_removed[int(index_cmin)]
            for i in range(0,PopSize-1):
                cd[0,i]=self.Dissimilarity(pop_cmin_removed[i])           
        return np.min(cd)
    
    
    def __str__(self):
        return str((self.Chromosome,self.Fitness))
    
def FunctionEvaluation(f,ListOfVars,ListOfVals):
        Dim=len(ListOfVars)
        for i in range(0,Dim):
            f=f.subs({ListOfVars[i]:ListOfVals[i]})
        return f

In [ ]:
import numpy as np
import sympy
import random

#s = str('x1**2+sin(x2)')
#ListOfVars=['x1','x2']
#f = sympy.sympify(s)

#Function='x1^2*sin(x2)+cos(x3)'
#ListOfVars=['x1','x2','x3']
#R=np.array([[-1,1],[-1,1],[-1,2]])

Function='sin(x1)'
ListOfVars=['x1']
R=np.array([[-np.pi,np.pi]])


NumGen=10
PopSize=50

#Function='x1^2+1'
#ListOfVars=['x1']
#R=np.array([[-1,1]])
#NumGen=500
#PopSize=50

In [ ]:
s=str(Function)
f=sympy.sympify(s)
dim=len(ListOfVars)

Population=[]

Fitness=np.empty(shape=[1, PopSize]) 
MinFit=np.empty(shape=[1, NumGen])
MaxFit=np.empty(shape=[1, NumGen])
AverageFit=np.empty(shape=[1, NumGen])
        
for i in range(0,PopSize):#2
    p=Individual()
    p.Create(dim,R)
    Population.append(p)  #.Chromosome

    ListofVals=p.Chromosome[0]
    Fitness[0,i]=FunctionEvaluation(f,ListOfVars,np.ndarray.tolist(ListofVals))
        
for n in range(0,NumGen):
    index1=random.randint(0,PopSize-1)
    index2=random.randint(0,PopSize-1)
        
    parent1=Population[index1]
    parent2=Population[index2]
        
    fitness_parent1=Fitness[0,index1]
    fitness_parent2=Fitness[0,index2]
        
    child1, child2= parent1.Crossover(parent2)
        
    child1=child1.Mutation(R,dim)
        #child2=child2.Mutation(R,dim)
        
    
    ListofVals=child1.Chromosome[0]
    fitness_child1=FunctionEvaluation(f,ListOfVars,np.ndarray.tolist(ListofVals))
        #ListofVals=child2.Chromosome[0]
        #fitness_child2=FunctionEvaluation(f,ListOfVars,np.ndarray.tolist(ListofVals))
        
        
        #DCRW
        
    counter=0;
    contribution_diversity_worse=[]
    contribution_diversity_worse_indices=[]
    for i in range(0,PopSize):
        if fitness_child1<Fitness[0,i]:
            counter=counter+1;
            contribution_diversity_worse=np.hstack([contribution_diversity_worse, Population[i].contribution_diversity(i,Population,PopSize,"Regualr")])
            contribution_diversity_worse_indices=np.hstack([contribution_diversity_worse_indices, i])
        
    if counter==0:   # replace worse
        index=np.argmax(Fitness)
        Population[index]=child1;
        Fitness[0,index]=fitness_child1;
           
           
    if counter!=0:   #Case 1
        index_cmin=np.argmin(contribution_diversity_worse)
        cmin_contribution_diversity=np.min(contribution_diversity_worse)
        index_cmin=contribution_diversity_worse_indices[index_cmin]
        cmin=Population[int(index_cmin)]
            
            #now compare contr_diver of this elemet to the contr_div of the child to the pop (removing cmin from pop)
             #find the con_divversity of child to the pop (where cmin removed frm pop
                                     
        #pop_cmin_removed=Population
        #del pop_cmin_removed[int(index_cmin)]
        contribution_diversity_child = child1.contribution_diversity(index_cmin,Population,PopSize,"cmin-removed")
        #contribution_diversity_child=Population[i].contribution_diversity(PopSize,Population,PopSize)

        if contribution_diversity_child>cmin_contribution_diversity:
            Population[int(index_cmin)]=child1;
            Fitness[0,int(index_cmin)]=fitness_child1
            
        else: #replace worse
            
            index=np.argmax(Fitness)
            Population[index]=child1;
            Fitness[0,index]=fitness_child1;   
            
    

    MinFit[0,n]=np.min(Fitness)
    MaxFit[0,n]=np.max(Fitness)
    AverageFit[0,n]=np.average(Fitness)
        
        
    if MinFit[0,n]==MaxFit[0,n]:
        break
        
    if np.isnan(np.max(MinFit)):
        break

a=np.min(Fitness)
b=np.ndarray.tolist(Population[np.argmin(Fitness)].Chromosome[0])

print("The minimim of function  ", f, ' is ', a, 'occured in  ', b)
    #return MinFit,MaxFit, AverageFit,a,b,n
    


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

t = np.arange(0.0, NumGen, 1)

plt.plot(t,MinFit[0,:])
plt.plot(t,MaxFit[0,:])
plt.plot(t,AverageFit[0,:])

#a=np.min(Fitness)
#b=np.ndarray.tolist(Population[np.argmin(Fitness)].Chromosome[0])

#print("The minimim of function  ", f, ' is ', a, 'occured in  ', b)

In [ ]:
plt.plot(t,MinFit[0,:])

In [ ]:
t=np.arange(0,100,10)
y=np.sin(t)
plt.plot(t,y,marker='o',linestyle='',color='r')


In [ ]:
for i in range(0,PopSize):
        plt.plot(Population[i].Chromosome,Fitness[0,i],marker='o',linestyle='',color='r')

t=np.arange(R[0,0],R[0,1],.1)
y=np.empty(shape=[1, len(t)]) 
for i in range(0,len(t)):
    y[0,i]=FunctionEvaluation(f,ListOfVars,np.array([t[i]]))
plt.plot(t,y[0,:])




In [ ]:
np.shape(t)

In [ ]:
np.shape(y)

In [ ]:
np.array([t[i]])